# Really tho'... how many ingredients could there possibly be???

In [47]:
# imports.  you're prolly gonna use all these, right?
import pdb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

import re
import string

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [25]:
# Pulling in recipe lists
df = pd.read_csv('../data/clean_recipes.csv', sep=';')
kgl1_df = pd.read_json('../data/kgl_ingredient_train.json')
kgl2_df = pd.read_json('../data/kgl_ingredient_test.json')

In [26]:
df = df.rename(columns={'Ingredients':'ingredients'})
df.head(3)

,Recipe Name,Review Count,Recipe Photo,Author,Prepare Time,Cook Time,Total Time,ingredients,Directions,RecipeID
0,Golden Crescent Rolls Recipe,304,https://images.media-allrecipes.com/userphotos...,Mike A.,25 m,15 m,3 h 10 m,"yeast,water,white sugar,salt,egg,butter,flour,...","Dissolve yeast in warm water.**Stir in sugar, ...",7000
1,Poppy Seed Bread with Glaze Recipe,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,15 m,1 h,1 h 20 m,"flour,salt,baking powder,poppy,butter,vegetabl...",'Preheat oven to 350 degrees F (175 degrees C)...,7001
2,Applesauce Bread I Recipe,124,https://images.media-allrecipes.com/userphotos...,GAF55,10 m,1 h 20 m,1 h 30 m,"flour,egg,white sugar,vegetable oil,applesauce...",Preheat oven to 350 degrees F (175 degrees C)....,7003


In [27]:
# df ingredients is a single string, splitting into a list of strings
df['ingredients'] = df['ingredients'].apply(lambda x: x.split(','))

In [43]:
l1 = df['ingredients']
l2 = kgl1_df['ingredients']
l3 = kgl2_df['ingredients']

ingr_series = l1.append([l2, l3], ignore_index=True)

In [44]:
ingr_series[:10]

0    [yeast, water, white sugar, salt, egg, butter,...
1    [flour, salt, baking powder, poppy, butter, ve...
2    [flour, egg, white sugar, vegetable oil, apple...
3    [flour, baking powder, baking soda, salt, cinn...
4    [oat, buttermilk, vegetable oil, egg, brown su...
5    [shortening, white sugar, salt, milk, egg, lem...
6    [shortening, water, brown sugar, yeast, water,...
7    [water, cottage cheese, vegetable oil, egg, fl...
8    [margarine, white sugar, egg, banana, coffee, ...
9    [cornmeal, milk, egg, bell pepper, onion, garl...
Name: ingredients, dtype: object

In [163]:
def scrub_ingredients(ingr_list):
    
    chtble = str.maketrans('', '', string.punctuation + '0123456789')
    
    # drop special characters and multiple spaces, then convert to lowercase
    clean_list = []
    for ingr_string in ingr_list:
        ingr_string = re.sub(r'\W\s+', ' ', ingr_string.strip().translate(chtble))
        clean_list.append(ingr_string.lower())

    # convert to lowercase & drop known measurement words
    dropwords = ['ounce', 'ounces', 'oz', 'lb', ' pound', 'the', 'a', 'an'
                 'package', 'packages', 'inches', 'and', 'for', 'as', 'is']

    outlist = []

    for i in clean_list:
        words = i.split()
        words = [word for word in words if word not in dropwords]
        i = ' '.join(words)
        outlist.append(i)
    
    return outlist

In [164]:
ingr_s = ingr_series.apply(scrub_ingredients)
ingr_s[:10]

0    [yeast, water, white sugar, salt, egg, butter,...
1    [flour, salt, baking powder, poppy, butter, ve...
2    [flour, egg, white sugar, vegetable oil, apple...
3    [flour, baking powder, baking soda, salt, cinn...
4    [oat, buttermilk, vegetable oil, egg, brown su...
5    [shortening, white sugar, salt, milk, egg, lem...
6    [shortening, water, brown sugar, yeast, water,...
7    [water, cottage cheese, vegetable oil, egg, fl...
8    [margarine, white sugar, egg, banana, coffee, ...
9    [cornmeal, milk, egg, bell pepper, onion, garl...
Name: ingredients, dtype: object

In [166]:
def ingredient_count(dfcolumn):

    # Create dictionary with word count
    d = dict()

    for l in dfcolumn:
        for i in l:
            if i in d:
                d[i] = d[i] + 1
            else:
                d[i] = 1

    return pd.DataFrame(d.items(), columns=['ingredient', 'count'])

In [170]:
# create df of unique ingredients and their number of occurances in recipe dfs
ingr_count_df = ingredient_count(ingr_s).sort_values('count', ascending=False).reset_index(drop=True)
ingr_count_df.head(15)

,ingredient,count
0,salt,27050
1,water,11899
2,garlic,11287
3,butter,10534
4,onions,10008
5,olive oil,9889
6,sugar,9319
7,garlic cloves,7772
8,pepper,7080
9,vegetable oil,6874


In [171]:
modeling_df = ingr_s.apply(pd.Series)
modeling_df = modeling_df.rename(columns = lambda x: 'ingr_' + str(x))
modeling_df.head(10)

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,yeast,water,white sugar,salt,egg,butter,flour,butter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,flour,salt,baking powder,poppy,butter,vegetable oil,egg,milk,white sugar,vanilla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,flour,egg,white sugar,vegetable oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,flour,baking powder,baking soda,salt,cinnamon,nutmeg,brown sugar,oat,apple,walnut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,oat,buttermilk,vegetable oil,egg,brown sugar,flour,baking powder,baking soda,salt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,shortening,white sugar,salt,milk,egg,lemon,yeast,flour,fruit,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,shortening,water,brown sugar,yeast,water,white sugar,salt,bread,whole wheat,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,water,cottage cheese,vegetable oil,egg,flour,white sugar,baking soda,salt,yeast,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,margarine,white sugar,egg,banana,coffee,water,vanilla,flour,salt,baking powder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,cornmeal,milk,egg,bell pepper,onion,garlic,salt,baking soda,white sugar,corn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
(modeling_df.isnull().mean() * 100).head(25)

ingr_0      0.000000
ingr_1      0.069278
ingr_2      0.667000
ingr_3      2.571332
ingr_4      6.173774
ingr_5     11.951216
ingr_6     19.618489
ingr_7     28.648762
ingr_8     38.076979
ingr_9     47.830640
ingr_10    57.033302
ingr_11    65.470686
ingr_12    72.807682
ingr_13    78.997567
ingr_14    84.048398
ingr_15    87.987562
ingr_16    91.190449
ingr_17    93.689281
ingr_18    95.559780
ingr_19    96.872835
ingr_20    97.902334
ingr_21    98.538723
ingr_22    98.980167
ingr_23    99.284667
ingr_24    99.487667
dtype: float64

In [178]:
ingr_int = {ingr:i for i, ingr in enumerate(ingr_count_df['ingredient'])}
int_ingr = {i:ingr for i, ingr in enumerate(ingr_count_df['ingredient'])}

In [207]:
# convert ingredients to numeric representations
encoded_df = modeling_df.applymap(ingr_int.get)
encoded_df.head()

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,352,1,10,0,15,3,12,3,4232,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
1,12,0,27,1089,3,9,15,14,10,25,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
2,12,15,10,9,679,226,44,35,27,31,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
3,12,27,35,0,44,131,24,278,253,122,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
4,278,90,9,15,24,12,27,35,0,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232


In [212]:
# drop all recipes that have fewer than 5 ingredients
encoded_df = encoded_df[encoded_df['ingr_5'] < 4232]
encoded_df.reset_index(drop=True)

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,352,1,10,0,15,3,12,3,4232,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
1,12,0,27,1089,3,9,15,14,10,25,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
2,12,15,10,9,679,226,44,35,27,31,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
3,12,27,35,0,44,131,24,278,253,122,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
4,278,90,9,15,24,12,27,35,0,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54222,338,323,138,129,21,8,94,97,80,119,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
54223,163,50,6,422,571,189,0,2795,416,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
54224,16,0,273,494,163,314,1483,9,4232,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
54225,214,194,56,117,78,1,457,39,730,14,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232


In [195]:
modeling_df = modeling_df.replace(np.nan, '', regex=True)
modeling_df.head()

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,yeast,water,white sugar,salt,egg,butter,flour,butter,,,...,,,,,,,,,,
1,flour,salt,baking powder,poppy,butter,vegetable oil,egg,milk,white sugar,vanilla,...,,,,,,,,,,
2,flour,egg,white sugar,vegetable oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,...,,,,,,,,,,
3,flour,baking powder,baking soda,salt,cinnamon,nutmeg,brown sugar,oat,apple,walnut,...,,,,,,,,,,
4,oat,buttermilk,vegetable oil,egg,brown sugar,flour,baking powder,baking soda,salt,,...,,,,,,,,,,


In [ ]:
# create sequences for model to train on
maxlen = 5
step = 2

sequences = []
next_ingr = []